In [1]:
# Open file and parse contents
file = open('day13_inputs.txt')
content = [line.strip() for line in file]

content

['1002394',
 '13,x,x,41,x,x,x,37,x,x,x,x,x,419,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,19,x,x,x,23,x,x,x,x,x,29,x,421,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,17']

Input consists of 2 lines:
- The earliest time you can depart (in minutes from a start time of 0 minutes)
- A list of bus IDs

Each bus' ID represents the frequency of the bus, for example: bus 7 would depart at 0, 7, 14, 21 etc. minutes, bus 13 at 0, 13, 26, 39 etc. minutes.

An 'x' infers that you can ignore that bus.

### Part 1

We want to find the **earliest** bus that we can take.

For a departure time of >=939 minutes, and buses [7,13,59,31,19], the earliest bus would be the 59 at 944 minutes (59 * 16).

We want the minutes waited (944 - 939) multiplied by the bus ID (59), 5 * 59 = 295.

In [2]:
earliest_dept = int(content[0])
bus_ids = [int(bus_id) for bus_id in content[1].split(',') if bus_id != 'x']

earliest_dept, bus_ids

(1002394, [13, 41, 37, 419, 19, 23, 29, 421, 17])

In [3]:
import itertools

dept_time = earliest_dept
earliest_bus_id = None

# Repeat loop until we have a valid number
for bus_id_list in itertools.repeat(bus_ids,10):
    
    # Check each bus_id
    for bus_id in bus_id_list:
        
        if dept_time % bus_id == 0:
            earliest_bus_id = bus_id
    
    
    if earliest_bus_id is not None:
        wait_time = dept_time-earliest_dept
        print('Earliest bus {}, departing at {}, wait time {}, puzzle answer {}'.format(earliest_bus_id,
                                                                                        dept_time,
                                                                                        wait_time,
                                                                                        wait_time*earliest_bus_id))
        break
    else:
        dept_time+=1

Earliest bus 421, departing at 1002401, wait time 7, puzzle answer 2947


### Part 2

Find the time at which each bus in the list departs 1 minute after the prior bus.

e.g. for: 7,13,x,x,59,x,31,19, find the time t at which:
- Bus 7 departs at t
- Bus 13 departs at t+1
- Bus 59 departs at t+4
- Bus 31 departs at t+6
- Bus 19 departs at t+7

Disregard the first line of our input (earliest departure time).

x's still don't matter (it could be any bus ID), but the position of our other bus IDs do.

In [4]:
content_list = content[1].split(',')
bus_ids_with_pos = [[int(content_list[i]), i] for i in range(len(content_list)) if content_list[i] != 'x']
bus_ids_with_pos

[[13, 0],
 [41, 3],
 [37, 7],
 [419, 13],
 [19, 32],
 [23, 36],
 [29, 42],
 [421, 44],
 [17, 61]]

In [41]:
n = 526090562196173
for b, o in bus_ids_with_pos:
    print(b, o, (n+o)%b)

13 0 0
41 3 0
37 7 0
419 13 0
19 32 0
23 36 0
29 42 0
421 44 0
17 61 0


All bus IDs are prime numbers, so we can do the following:
- Loop through numbers, 1 at a time, to find a timestamp T that satisfies the first 2 cases (i.e. timestamp divisible by 13, and when 3 is added is also divisible by 41)
- Since 13 & 41 are co-primes (their only common divisor is 1), the next timestamp TX that will satisfy these 2 cases again will be a multiple of 13 * 41 (533) away - i.e. TX = T + n(533)

Once we have this as a starting point:
- We then add on 533 until we find a timestamp T2 that satisfies the third ID, when 7 is added it's divisible by 13
- Then, again knowing that our next timestamp T3 that will now satisfy our first **three** IDs will be a multiple of 13 * 41 * 37 (19,721) - i.e. T3 = T2 + m(19,721) = T + n(533) + m(19,721)

We can then repeat these last 2 steps until we find a timestamp that satisfies all IDs.

In [39]:
# Figure out starting point
first_id = bus_ids_with_pos[0][0]
second_id = bus_ids_with_pos[1][0]
second_id_offset = bus_ids_with_pos[1][1]
t = 0

# Get starting t
while not (t % first_id == 0 and (t + second_id_offset) % second_id == 0):
    t+=1
    
print('First t that divides by {} and, when {} is added, divides {}, is {}'.format(first_id,
                                                                                   second_id_offset,
                                                                                   second_id,
                                                                                   t))

# Must jump by multiples of first_id * second_id to reach each valid timestamp after t_init that
# Satisfies the t and t+1 requirements
int_jump = first_id * second_id
        
# Loop through rest of list
for bus_id, offset in bus_ids_with_pos[2:]:
    
    # Find timestamp that also satisfies next id in list
    while (t+offset) % bus_id != 0:
        t+=int_jump
    
    print('Loop for ID {} and offset {} finished, timestamp {}'.format(bus_id,
                                                                       offset,
                                                                       t))
    # Must then jump in integers that are multiple of this bus_id
    int_jump*=bus_id   

First t that divides by 13 and, when 3 is added, divides 41, is 325
Loop for ID 37 and offset 7 finished, timestamp 2990
Loop for ID 419 and offset 13 finished, timestamp 4085237
Loop for ID 19 and offset 32 finished, timestamp 28874534
Loop for ID 23 and offset 36 finished, timestamp 499871177
Loop for ID 29 and offset 42 finished, timestamp 90774227752
Loop for ID 421 and offset 44 finished, timestamp 41140329649536
Loop for ID 17 and offset 61 finished, timestamp 526090562196173
